In [124]:
'''import cv2
import numpy as np
from skimage import io, color
import matplotlib.pyplot as plt
%matplotlib inline


def get_sift_feature(im):
    """
    Extract the SIFT features from an input image.

    Parameters:
        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).

    Returns:
        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,
        returns a zero array of length 128.
    """
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0)



img = cv2.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bae8f.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)


img1 = io.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bb10e.jpg")
io.imshow(img1)
io.show()
print(get_sift_feature(img1))'''

'import cv2\nimport numpy as np\nfrom skimage import io, color\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n\ndef get_sift_feature(im):\n    """\n    Extract the SIFT features from an input image.\n\n    Parameters:\n        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).\n\n    Returns:\n        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,\n        returns a zero array of length 128.\n    """\n    gray_im = color.rgb2gray(im)\n    sift = cv2.SIFT_create()\n    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)\n    if descriptors is None:\n        return np.zeros((128,))\n    return np.mean(descriptors, axis=0)\n\n\n\nimg = cv2.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bae8f.jpg")\nimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\nplt.imshow(img)\n\n\nimg1 = io.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP

In [125]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage import io, color
from skimage.transform import resize
from skimage.feature import hog
#from sklearn.feature_extraction import image
#from sklearn.feature_extraction.image import extract_surf_features
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

# Get the directory path
directory_path = "/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train_ims"

# List filenames in alphabetical order
filenames = sorted(os.listdir(directory_path))

In [126]:
def get_sift_feature(im):
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0) 

def get_orb_feature(im):
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    _, descriptors = orb.detectAndCompute(gray_im, None)

    if descriptors is None:
        return np.zeros((32,))

    # Calculate the mean of the ORB descriptors
    mean_descriptor = np.mean(descriptors, axis=0)
    return mean_descriptor

def get_hog_feature(im):
    gray_im = color.rgb2gray(im)
    hog_features = hog(gray_im, orientations=4, pixels_per_cell=(4, 4), cells_per_block=(1, 1))
    hog_features = hog_features / np.sum(np.abs(hog_features))
    return(hog_features)

In [127]:
initial_images = []
train_df = pd.read_csv("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train.csv") #change path
labels = train_df["label"].to_numpy()[:1000] #sliced to 1000


for i in range(1000): #only checking first 1000 images
    image = io.imread(f"{directory_path}/{filenames[i]}")
    initial_images.append(image)

In [128]:
sift_features = []
for image in initial_images:
    sift_features.append(get_sift_feature(image)) #checking for sift feature instead

sift_features = np.array(sift_features)
labels = np.array(labels)

scaler = MinMaxScaler(copy=False)
scaler.fit(sift_features)
sift_features_scaled = scaler.transform(sift_features)

print(sift_features_scaled)

pca = PCA(n_components=128, copy=False)
pca.fit(sift_features_scaled)
sift_features_reduced = pca.transform(sift_features_scaled)


param_grid = {'C': [0.01, 0.1, 1, 5], 'kernel': ['linear', 'rbf', 'poly'], 'gamma':[0.1, 0.2, 0.4, 0.5]}
svc = SVC()
clf = GridSearchCV(svc, param_grid, cv=4)

# Train the SVM classifier
clf.fit(sift_features_reduced, labels)

# Print the best hyperparameters and score
print("Best parameters:", clf.best_params_)
print("Best score:", clf.best_score_)

[[0.42313663 0.62613428 0.28346458 ... 0.04448838 0.07108681 0.33677348]
 [0.02536232 0.09739867 0.42125987 ... 0.85351787 0.52312601 0.10139417]
 [0.22306239 0.31484257 0.15405683 ... 0.23845345 0.14645309 0.30087617]
 ...
 [0.25603865 0.34018956 0.3589239  ... 0.08746636 0.22753855 0.28136882]
 [0.23136647 0.22504536 0.15804275 ... 0.25786786 0.22693096 0.16657614]
 [0.21086956 0.19649122 0.25826773 ... 0.23298731 0.22838916 0.07503169]]
Best parameters: {'C': 1, 'gamma': 0.4, 'kernel': 'rbf'}
Best score: 0.298


In [130]:
hog_features = []
for image in initial_images:
    hog_features.append(get_hog_feature(image))

hog_features = np.array(hog_features)

scaler.fit(hog_features)
hog_features_scaled = scaler.transform(hog_features)

print(hog_features_scaled)

param_grid = {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma':[0.1, 0.2, 0.4, 0.5, 0.6]}
svc = SVC()
clf = GridSearchCV(svc, param_grid, cv=4)

# Train the SVM classifier
clf.fit(hog_features_scaled, labels)

# Print the best hyperparameters and score
print("Best parameters:", clf.best_params_)
print("Best score:", clf.best_score_)

[[0.45843449 0.4662908  0.43506592 ... 0.50334876 0.41886333 0.59531093]
 [0.5179575  0.52563398 0.49168475 ... 0.5304493  0.44141514 0.41263663]
 [0.53568956 0.54331246 0.50855165 ... 0.50791665 0.42266452 0.60071338]
 ...
 [0.14731525 0.676889   0.63599593 ... 0.52349707 0.43562981 0.61914034]
 [0.48214595 0.48993063 0.45762048 ... 0.48426358 0.54253901 0.2185968 ]
 [0.53729267 0.54491073 0.51007655 ... 0.51504091 0.428593   0.5820474 ]]
Best parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Best score: 0.36100000000000004
